#### Agents SDK Course

## Handoffs

Agents SDK introduces a few unique selling points, one of these is the handoff functionallity, other frameworks refer to this as multiple agentic systems but tend to provide tools where within the agent is, however for this framework you can directly link another agent to a pre-existing agent without the fuss, and alongside this the devs have provided formatting tools to provide a depth of functionallity to your agentic systems.

First we need to get an `OPENAI_API_KEY`, this will be used to communicate with LLMs, to get one head to the [OpenAI API keys website](www.openai.com/api-keys)

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or \
    getpass.getpass("OpenAI API Key: ")

Next we need to define the context for the agents, this will be used to provide the agents with the information they need to answer the questions and make each agent more specific for the queries they will be used for.

In [2]:
drum_context = """
below is a list of drum products and their prices

beginners kit = 400
advanced kit = 600
snare drum = 100
bass drum = 150
cymbal = 200

during happy hour (7-8pm) all prices are 50% off
"""

guitar_context = """
below is a list of guitar products and their prices

electric guitar = 1000
acoustic guitar = 500
bass guitar = 700
guitar strings = 10
guitar picks = 5
guitar strap = 20
guitar case = 50
guitar amp = 100
guitar pedal = 20

during happy hour (7-8pm) all prices are 50% off
"""

Now we need to create `Agent` objects, first we will create the bottom layer agents that will be used to help with specific details, in this example we will create two agents one for guitars and one for drums, that will contain the context created earlier to help with the queries they will be used for.

In each agent we have the following:
- **name**: the name of the agent, ie `Guitar agent` or `Drums agent`
- **model**: the model to use for the agent - for the bottom layer agents we will use the `gpt-4o-mini` model
- **instructions**: the instructions to use for the agent - we will provide the context written earlier here too!


In [3]:
from agents import Agent

# bottom layer agent
guitar_agent = Agent(
    name="Guitar agent",
    model="gpt-4o-mini",
    instructions=(
        "You are responsible for dealing with guitar information and transactions, always start by "
        f"saying 'Hi this is Tim from guitar' {guitar_context}"
    )
)

# bottom layer agent
drums_agent = Agent(
    name="Drums agent",
    model="gpt-4o-mini",
    instructions=(
        "You are responsible for dealing with drum information and transactions, always start by "
        "saying 'Hi this is Steve from drums' {drum_context}"
    )
)

Next we need to create the top layer agent that will be used to orchestrate the `handoff`, this agent contains additonal properties such as:
- **handoffs**: the handoffs to use for the agent - we will provide the bottom layer agents in here
- **handoff_description**: the description of the handoff - this will be used to provide the AI with information about when to handoff etc...

We can also import the `RECOMMENDED_PROMPT_PREFIX` from the `handoff_prompt` extension, this will provide the agent with the correct formatting for the handoff defined by the devs.

In [4]:
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX

# top layer agent
orchestration_agent = Agent(
    name="Orchestration agent", 
    handoffs=[drums_agent, guitar_agent],
    handoff_description="For any queries about guitars or drums, handoff to the relevant agent",
    model="gpt-4o",
    instructions=RECOMMENDED_PROMPT_PREFIX
)

Now using the `Runner` we apply the `run` method to the orchestration agent, this requires us to define the `starting_agent` and the `input` to the agent.

In [5]:
from agents import Runner

result = await Runner.run(
    starting_agent=orchestration_agent,
    input="I want to prices and information about the guitar"
)

Using the `run` method provides us with a `RunResult` object, this contains a list of useful properties listed below:
- **input**: the input to the agent
- **new_items**: this provides a `HandoffCallItem` object that details all the agents and their associated properties involved in the orchestration agent
- **raw_responses**: contains the raw model responses
- **final_output**: the final output from the orchestration agent
- **input_guardrail_results** and **output_guardrail_results**: these contain the guardrail results for the input and output of the agent
- **_last_agent**: this contains the last agent that was called

The attributes that we are interested in for this tutorial are the `new_items`, `raw_responses`, `final_output` and `_last_agent`.

In [6]:
print("Item details: ", result.new_items) # details about the handoff
print("Raw responses: ", result.raw_responses) # the raw model responses
print("Final output: ", result.final_output) # the final output from the run
print("Last agent called: ", result._last_agent) # the last agent that was called

Item details:  [HandoffCallItem(agent=Agent(name='Orchestration agent', instructions='# System context\nYou are part of a multi-agent system called the Agents SDK, designed to make agent coordination and execution easy. Agents uses two primary abstraction: **Agents** and **Handoffs**. An agent encompasses instructions and tools and can hand off a conversation to another agent when appropriate. Handoffs are achieved by calling a handoff function, generally named `transfer_to_<agent_name>`. Transfers between agents are handled seamlessly in the background; do not mention or draw attention to these transfers in your conversation with the user.\n', handoff_description='For any queries about guitars or drums, handoff to the relevant agent', handoffs=[Agent(name='Drums agent', instructions="You are responsible for dealing with drum information and transactions, \n        always start by saying 'Hi this is Steve from drums'\n        \nbelow is a list of drum products and their prices\n\nbegin

We can then take this a step futher and create a `on_handoff` function that will be called when the handoff is made, this can be useful for noting down the handoff in a database or simply making routine tasks before the handoff is made.

In [7]:
from agents import RunContextWrapper

async def on_handoff(ctx: RunContextWrapper[None]):
    print(f"Handoff Called!")

Now we will redefine the orchestration agent, this time we will provide a `handoff` object instead of the agents directly, this allows us to provide additional information such as the `on_handoff` function.

In [8]:
from agents import handoff

# top layer agent
orchestration_agent = Agent(
    name="Orchestration agent",
    model="gpt-4o",
    instructions=RECOMMENDED_PROMPT_PREFIX,
    handoff_description="For any queries about guitars, handoff to the guitar agent",
    handoffs=[
        handoff(
            agent=guitar_agent,
            on_handoff=on_handoff
        )
    ]
)

Now we can test our function we made earlier, we will run the orchestration agent, and if the handoff is made we should see the print statement we made earlier.

In [9]:
result = await Runner.run(
    starting_agent=orchestration_agent,
    input="I want to prices and information about the guitar"
)

Handoff Called!


We can take this a step further and create dynamic handoffs, that can encapsulate data and print out to provide reasoning for the handoff.

For this we need to create a class that inherits from `BaseModel` from `pydantic`, this is required to pass the `__pydantic_validator__` attribute in the handoff functionallity.


In [10]:
from pydantic import BaseModel

class handoff_data(BaseModel):
    reason: str

async def on_handoff(ctx: RunContextWrapper[None], input_data: handoff_data):
    print(f"Handoff called with reason: {input_data.reason}")

The only noteable difference here is the `input_type` parameter, this is used to pass the class we created earlier to the handoff function, this will allow the top layer agent to pass data required to the bottom layer agents.

In [11]:
# top layer agent
orchestration_agent = Agent(
    name="Orchestration agent",
    model="gpt-4o",
    instructions=RECOMMENDED_PROMPT_PREFIX,
    handoff_description="For any queries about guitars, handoff to the guitar agent",
    handoffs=[
        handoff(
            agent=guitar_agent,
            on_handoff=on_handoff,
            input_type=handoff_data
        )
    ]
)

As before if we run the orchestration agent we can see the handoff is made and the reason is printed out.

In [12]:
result = await Runner.run(
    starting_agent=orchestration_agent,
    input="I want to prices and information about the guitar"
)

Handoff called with reason: Inquiry about guitar prices and information.


We can also take a look at how to include / exclude tools from the handoff, this can be useful for when we want to handoff to an agent but not provide them with the tools that a top layer agent has.

First we need to create a function with the `@function_tool` decorator, this will allow the function to be used as a tool in our orchestration agent.

In [13]:
from agents import function_tool

@function_tool
def get_current_time():
    return "time is 7pm"

Next we want to include this tool in our orchestration agent, we can do this by adding the tool to the `tools` parameter in the orchestration agent.

In [14]:
# top layer agent
orchestration_agent = Agent(
    name="Orchestration agent",
    model="gpt-4o",
    instructions=RECOMMENDED_PROMPT_PREFIX,
    tools=[get_current_time],
    handoff_description="For any queries about guitars, handoff to the guitar agent",
    handoffs=[
        handoff(
            agent=guitar_agent,
            on_handoff=on_handoff,
            input_type=handoff_data
        )
    ]
)

If we run this currently we can see the handoff is made...

In [15]:
result = await Runner.run(
    starting_agent=orchestration_agent,
    input="On the current time, how much is an electric guitar?"
)

Handoff called with reason: Get the current price of an electric guitar.


And currently the tool is available to the guitar agent.

In [16]:
print("Final output: ", result.final_output)

Final output:  Hi, this is Tim from guitar. Since it's currently happy hour (7-8 PM), the price of an electric guitar is 50% off. That makes it $500. If you have any more questions or need help with something else, feel free to ask!


If we want to exclude the top layer agent's tools from bottom layer agents we can use the `input_filter` parameter in the handoff function. We can then pass the `remove_all_tools` function from the `handoff_filters` extension. This will remove all tools from the bottom layer agent that was passed via the top agent.

In [17]:
from agents.extensions import handoff_filters

# top layer agent
orchestration_agent = Agent(
    name="Orchestration agent",
    model="gpt-4o",
    instructions=RECOMMENDED_PROMPT_PREFIX,
    tools=[get_current_time],
    handoff_description="For any queries about guitars, handoff to the guitar agent",
    handoffs=[
        handoff(
            agent=guitar_agent,
            on_handoff=on_handoff,
            input_type=handoff_data,
            input_filter=handoff_filters.remove_all_tools
        )
    ]
)

Now if we run the agent again we should see the tool is not available to the guitar agent.

In [18]:
result = await Runner.run(
    starting_agent=orchestration_agent,
    input="On the current time, how much is an electric guitar?"
)

Handoff called with reason: User wants to know the price of an electric guitar.


And as we can see the tool is not available to the guitar agent.

In [19]:
print("Final output: ", result.final_output)

Final output:  Hi, this is Tim from guitar. The price of an electric guitar is $1000. If you're looking for a discount, remember that during happy hour (7-8pm), all prices are 50% off!


If you would like a visualization of the graph the agents sdk team have viz dependency group which allows developers to view the graph - this doesnt work without:

**pip install "openai-agents[viz]"**

In [ ]:
# from agents.extensions.visualization import draw_graph

# draw_graph(orchestration_agent)